# Traffic Violation RAG System
In this exam, you will implement a Retrieval-Augmented Generation (RAG) system that uses a language model and a vector database to answer questions about traffic violations. The goal is to generate answers with relevant data based on a dataset of traffic violations and fines.

Here are helpful resources:
* [LangChain](https://www.langchain.com/)
* [groq cloud documentation](https://console.groq.com/docs/models)
* [LangChain HuggingFace](https://python.langchain.com/docs/integrations/text_embedding/sentence_transformers/)
* [Chroma Vector Store](https://python.langchain.com/docs/integrations/vectorstores/chroma/)
* [Chroma Website](https://docs.trychroma.com/getting-started)
* [ChatGroq LangChain](https://python.langchain.com/docs/integrations/chat/groq/)
* [LLM Chain](https://api.python.langchain.com/en/latest/chains/langchain.chains.llm.LLMChain.html#langchain.chains.llm.LLMChain)

Dataset [source](https://www.moi.gov.sa/wps/portal/Home/sectors/publicsecurity/traffic/contents/!ut/p/z0/04_Sj9CPykssy0xPLMnMz0vMAfIjo8ziDTxNTDwMTYy83V0CTQ0cA71d_T1djI0MXA30gxOL9L30o_ArApqSmVVYGOWoH5Wcn1eSWlGiH1FSlJiWlpmsagBlKCQWqRrkJmbmqRqUZebngB2gUJAKdERJZmqxfkG2ezgAhzhSyw!!/)

Some installs if needed:
```python
!pip install langchain-huggingface langchain langchain-community langchain-chroma Chroma langchain-groq
```

In [ ]:
!pip install langchain-huggingface langchain langchain-community langchain-chroma Chroma langchain-groq

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 64.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 599.2/599.2 kB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 73.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 405.1/405.1 kB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.0/290.0 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
pip install chromadb


In [ ]:
!kaggle datasets download -d khaledzsa/dataset
!unzip dataset.zip

Dataset URL: https://www.kaggle.com/datasets/khaledzsa/dataset
License(s): unknown
dataset.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  dataset.zip
replace Dataset.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

## Step 1: Install Required Libraries

To begin, install the necessary libraries for this project. The libraries include `LangChain` for building language model chains, and `Chroma` for managing a vector database.

In [ ]:
import pandas as pd
import os
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain.prompts import PromptTemplate
from langchain_groq import ChatGroq
from langchain.chains import LLMChain




## Step 2: Load the Traffic Violations Dataset

You are provided with a dataset of traffic violations. Load the CSV file into a pandas DataFrame and preview the first few rows of the dataset using `.head()`. You can also try and see the dataset's characteristics.

In [ ]:
df = pd.read_csv('/content/Dataset.csv')
df.head()

,المخالفة,الغرامة
0,قيادة المركبة في الأسواق التي لا يسمح بالقيادة...,الغرامة المالية 100 - 150 ريال
1,ترك المركبة مفتوحة وفي وضع التشغيل بعد مغادرتها.,الغرامة المالية 100 - 150 ريال
2,عدم وجود تأمين ساري للمركبة.,الغرامة المالية 100 - 150 ريال
3,عبور المشاة للطرق من غير الأماكن المخصصة لهم.,الغرامة المالية 100 - 150 ريال
4,عدم تقيد المشاة بالإشارات الخاصة بهم.,الغرامة المالية 100 - 150 ريال


In [ ]:
Diroctry = 'Data/markeddowen_files'
os.makedirs(Diroctry,exist_ok=True)

## Step 3: Create Markdown Content from the Dataset

For each traffic violation in the dataset, you will generate markdown text that describes the violation and the associated fine. Create a loop to iterate through the dataset and store the generated markdown in a list. Each fine should look like this:

**المخالفة** - الغرامة

In [ ]:
markdown_files = []

for i in range(0,100):
  violation = df['المخالفة'].iloc[i]
  fine = df['الغرامة'].iloc[i]
  marked_dowen= f"**{violation}**-{fine}"
  markdown_files.append(marked_dowen)
  with open(f'{Diroctry}/{i}.md', 'w', encoding='utf-8') as file:
    file.write(marked_dowen)


In [ ]:
with open(os.path.join(Diroctry, '0.md'), 'r', encoding='utf-8') as file:
  markdown_content = file.read()
markdown_content

'**قيادة المركبة في الأسواق التي لا يسمح بالقيادة فيها.**-الغرامة المالية 100 - 150 ريال'

## Step 4: Chunk the Markdown Data

Using LangChain's `RecursiveCharacterTextSplitter`, split the markdown texts into smaller chunks that will be stored in the vector database.

In [49]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=5)
documents = text_splitter.create_documents(markdown_files)

## Step 5: Generate Embeddings for the Documents

Generate embeddings for the chunks of text using HuggingFace's pre-trained Arabic language model. These embeddings will be stored in a `Chroma` vector store.

In [50]:
embedding_function =  HuggingFaceEmbeddings(model_name='Omartificial-Intelligence-Space/GATE-AraBert-v0')
db = Chroma.from_documents(documents,embedding_function,persist_directory='./chroma_db')

In [51]:
documents

[Document(metadata={}, page_content='**قيادة المركبة في الأسواق التي لا يسمح بالقيادة فيها.**-الغرامة المالية 100 - 150 ريال'),
 Document(metadata={}, page_content='**ترك المركبة مفتوحة وفي وضع التشغيل بعد مغادرتها.**-الغرامة المالية 100 - 150 ريال'),
 Document(metadata={}, page_content='**عدم وجود تأمين ساري للمركبة.**-الغرامة المالية 100 - 150 ريال'),
 Document(metadata={}, page_content='**عبور المشاة للطرق من غير الأماكن المخصصة لهم.**-الغرامة المالية 100 - 150 ريال'),
 Document(metadata={}, page_content='**عدم تقيد المشاة بالإشارات الخاصة بهم.**-الغرامة المالية 100 - 150 ريال'),
 Document(metadata={}, page_content='**وقوف المركبة في أماكن غير مخصصة للوقوف.**-الغرامة المالية 100 - 150 ريال'),
 Document(metadata={}, page_content='**عدم إعطاء أفضلية المرور للمشاة أثناء عبورهم في المسارات المخصصة لهم.**-الغرامة المالية 100 - 150 ريال'),
 Document(metadata={}, page_content='**عدم استخدام إشارة الالتفاف عند التحول لليمين أو اليسار أو التجاوز أو تغيير المسار.**-الغرامة المالية 150 - 300 ر

In [ ]:
documents[0]

Document(metadata={}, page_content='**قيادة المركبة في')

In [52]:
res= db.similarity_search(" تأمين", k=4)

In [53]:
res

[Document(metadata={}, page_content='الأمان المخصصة'),
 Document(metadata={}, page_content='الاحتياطات'),
 Document(metadata={}, page_content='الاحتياطات اللازمة'),
 Document(metadata={}, page_content='الأمني أو نقاط')]

## Step 6: Define the RAG Prompt Template

Define a custom prompt template in Arabic to retrieve traffic violation-related answers based on the context. Ensure the template greets the user first, states that the information provided could be incorrect, and advises the user to visit the traffic initiative website to verify. Additionally, provide the user with advice in Arabic, ensuring it stays within the given context.

In [54]:
prompt_template = """
 اجب على السؤال بناء على السياق لكن تأكد ان ترحب بالمستخدم اولا ثم انصح المستخدم ان يزور موقع المرور للتاكد من المعلومة الصحيحةو ثم اجب على السؤال
---
السياق: {السياق}

---
السؤال: {السؤال}

---

اجابتك:

"""
prompt_template = PromptTemplate(
    template=prompt_template, input_variables=["السياق", "السؤال"]
)

## Step 7: Initialize the Language Model

Initialize the language model using the Groq API. Set up the model with a specific configuration, including the API key, temperature setting, and model name.

In [55]:
groq_api_key = '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'
llm = ChatGroq(temperature=0,
               groq_api_key=groq_api_key,
               model_name='llama-3.1-8b-instant')

## Step 8: Create the LLM Chain

Now, you will create an LLM Chain that combines the language model and the prompt template you defined. This chain will be used to generate responses based on the retrieved context.

In [56]:
 model = LLMChain(llm=llm,
                 prompt=prompt_template,
                         verbose=True)

In [57]:
res2 = db.similarity_search_with_relevance_scores("تأمين", k=2)

<ipython-input-57-42b20d099be3>:1: UserWarning: Relevance scores must be between 0 and 1, got [(Document(metadata={}, page_content='الأمان المخصصة'), -149.62319607639427), (Document(metadata={}, page_content='الاحتياطات'), -182.78442819630806)]
  res2 = db.similarity_search_with_relevance_scores("تأمين", k=2)


In [58]:
res2

[(Document(metadata={}, page_content='الأمان المخصصة'), -149.62319607639427),
 (Document(metadata={}, page_content='الاحتياطات'), -182.78442819630806)]

## Step 9: Implement the Query Function

Create a function `query_rag` that will take a user query as input, retrieve relevant context from the vector store, and use the language model to generate a response based on that context.

In [59]:
def query_rag(query: str):
  similarity_search_results =  db.similarity_search_with_relevance_scores(query)
  Context_text = ""
  for result in similarity_search_results:
    Context_text= Context_text + result[0].page_content+ '\n\n'
  rag_response = model.invoke({"السياق":Context_text, "السؤال": query})
  return rag_response

## Step 10: Inference - Running Queries in the RAG System

In this final step, you will implement an inference pipeline to handle real-time queries. You will allow the system to retrieve the most relevant violations and fines based on a user's input and generate a response.

1. Inference Workflow:

  * The user inputs a query (e.g., "ماهي عقوبة عدم الوقوف وقوفاً تاماً عند إشارة؟").
  * The system searches for the most relevant context from the traffic violation vector store.
  * It generates an answer and advice based on the context.

2. Goal:
  * Run the inference to answer questions based on the traffic violation dataset.

In [60]:
response = query_rag("ماهي عقوبة عدم الوقوف وقوفاً تاماً عند إشارة؟")

<ipython-input-59-69828d54c7ff>:2: UserWarning: Relevance scores must be between 0 and 1, got [(Document(metadata={}, page_content='**عدم التقيد بإشارات'), -191.00614138739806), (Document(metadata={}, page_content='**عدم الوقوف وقوفاً تاماً عند إشارة (أمامك أفضلية) في حالة مرور مركبات على الطريق المعطاة له الأفضلية.**-الغرامة المالية 500 - 900 ريال'), -226.6189077510661), (Document(metadata={}, page_content='**عدم الوقوف وقوفاً'), -227.055963403846), (Document(metadata={}, page_content='**عدم الوقوف تماماً'), -227.055963403846)]
  similarity_search_results =  db.similarity_search_with_relevance_scores(query)


Prompt after formatting:

 اجب على السؤال بناء على السياق لكن تأكد ان ترحب بالمستخدم اولا ثم انصح المستخدم ان يزور موقع المرور للتاكد من المعلومة الصحيحةو ثم اجب على السؤال
---
السياق: **عدم التقيد بإشارات

**عدم الوقوف وقوفاً تاماً عند إشارة (أمامك أفضلية) في حالة مرور مركبات على الطريق المعطاة له الأفضلية.**-الغرامة المالية 500 - 900 ريال

**عدم الوقوف وقوفاً

**عدم الوقوف تماماً



---
السؤال: ماهي عقوبة عدم الوقوف وقوفاً تاماً عند إشارة؟

---

اجابتك:



> Finished chain.


In [61]:
response

{'السياق': '**عدم التقيد بإشارات\n\n**عدم الوقوف وقوفاً تاماً عند إشارة (أمامك أفضلية) في حالة مرور مركبات على الطريق المعطاة له الأفضلية.**-الغرامة المالية 500 - 900 ريال\n\n**عدم الوقوف وقوفاً\n\n**عدم الوقوف تماماً\n\n',
 'السؤال': 'ماهي عقوبة عدم الوقوف وقوفاً تاماً عند إشارة؟',
 'text': 'مرحباً بك! أهلاً وسهلاً بكم على هذا الموقع. إذا كنت تبحث عن معلومات حول عقوبة عدم الوقوف وقوفاً تاماً عند إشارة، فإني أستعذبك من زيارة موقع المرور الرسمي لتحصل على المعلومات الصحيحة والمتجدة. \n\nومع ذلك، يمكنني أن أشاركك بعض المعلومات العامة حول هذا الموضوع. \n\nعقوبة عدم الوقوف وقوفاً تاماً عند إشارة في المملكة العربية السعودية هي غرامة مالية تتراوح بين 500 إلى 900 ريال. \n\nللحصول على المعلومات الصحيحة والمتجدة، أرجو زيارة موقع المرور الرسمي.'}